In [16]:
import pulp

def solve_held_karp(vertices, edges):
    prob = pulp.LpProblem("Held-Karp_Relaxation", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", [(i, j) for i, j, _ in edges], 
                              lowBound=0, upBound=1, cat=pulp.LpContinuous)

    prob += pulp.lpSum(c * x[i, j] for i, j, c in edges)

    for U in [set(vertices) - {v} for v in vertices]:
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if i in U and j not in U) >= 1
    
    for v in vertices:
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if i == v) == 1
        prob += pulp.lpSum(x[i, j] for i, j, _ in edges if j == v) == 1
    
    # Solve the LP
    prob.solve()
    
    # Extract results
    status = pulp.LpStatus[prob.status]
    total_cost = pulp.value(prob.objective)
    edge_solution = {(i, j): x[i, j].varValue for i, j, _ in edges}
    
    return status, total_cost, edge_solution

In [17]:
vertices = [0, 1, 2, 3]
edges = [
    (0, 1, 2), (1, 2, 2), (2, 0, 2),
    (0, 2, 1), (2, 1, 1), (1, 0, 1)
]

solve_held_karp(vertices, edges)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/chuck/miniforge3/envs/CS632/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/d610f5081f4144cea04bc7012bbf4258-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/d610f5081f4144cea04bc7012bbf4258-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 42 RHS
At line 55 BOUNDS
At line 62 ENDATA
Problem MODEL has 12 rows, 6 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 11.999999 (12)
3  Obj 3 Primal inf 2.9999997 (3)
Primal infeasible - objective value 3
PrimalInfeasible objective 3 - 3 iterations time 0.002

Result - Linear re

('Infeasible',
 3.0,
 {(0, 1): 0.0,
  (1, 2): 0.0,
  (2, 0): 0.0,
  (0, 2): 1.0,
  (2, 1): 1.0,
  (1, 0): 1.0})

In [15]:
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import held_karp_ascent

G = nx.complete_graph(3, create_using=nx.DiGraph)
nx.set_edge_attributes(G, {(0, 1): 2, (1, 2): 2, (2, 0): 2, (0, 2): 1, (2, 1): 1, (1, 0): 1}, "weight")

opt_hk, z_star = held_karp_ascent(G, 'weight')
print(opt_hk)
print(z_star.edges)

3.0
[(1, 0), (2, 1), (0, 2)]
